In [ ]:
from pyannote.audio import Model

In [ ]:
model = Model.from_pretrained("pyannote/embedding",
                              use_auth_token="AUTH_TOKEN")

In [ ]:
from pyannote.audio import Inference
import torch

inference = Inference(model, window="whole")
inference.to(torch.device("cuda"))

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("/content/ROBOVOX_SP_CUP_2024/data/single-channel/signle-channel-trials.trl", sep="\t", header=None)
df.head()

,0,1
0,spk_71,b80bb95d16
1,spk_71,e64da37b33
2,spk_71,c22d8f9d8a
3,spk_71,20e087d5bf
4,spk_71,c5396d4af7


In [ ]:
enrollment_path = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment"
file_list = [f for f in Path(enrollment_path).iterdir() if f.is_file()]
speaker_to_file_map = dict()

In [ ]:
for enrollment in file_list:
  speaker = enrollment.name.split('-')[0]
  array = speaker_to_file_map.get(speaker, -1)

  if(array==-1):
    speaker_to_file_map[speaker] = [str(enrollment)]
  else:
    speaker_to_file_map[speaker].append(str(enrollment))

In [ ]:
with open('output.txt', 'w') as output:
  for i in range(len(df)):
    speaker, test_file = df.iloc[i][0], df.iloc[i][1]

    test_embedding = inference(f"/content/ROBOVOX_SP_CUP_2024/data/single-channel/test/{test_file}.wav")

    enrollment_list = speaker_to_file_map.get(speaker)

    e_embedding_1 = inference(enrollment_list[0])
    e_embedding_2 = inference(enrollment_list[1])
    e_embedding_3 = inference(enrollment_list[2])

    average_embedding = np.mean([e_embedding_1, e_embedding_2, e_embedding_3], axis=0)

    test_embedding_normalized = test_embedding / np.linalg.norm(test_embedding)
    average_embedding_normalized = average_embedding / np.linalg.norm(average_embedding)

    similarity = np.dot(test_embedding_normalized, average_embedding_normalized) / ((np.dot(test_embedding_normalized, test_embedding_normalized) * np.dot(average_embedding_normalized, average_embedding_normalized)) ** 0.5)
    similarity = (similarity + 1) / 2

    # similarity = cosine_similarity(test_embedding.cpu(), average_embedding.cpu())[0,0]

    output.write(f"{speaker}\t{test_file}\t{similarity}\n")